# Azure ML Training Pipeline for COVID-CXR
This notebook defines an Azure machine learning pipeline for a single training run and submits the pipeline as an experiment to be run on an Azure virtual machine.

In [52]:
# Import statements
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace, Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.runconfig import RunConfiguration
import shutil

### Register the workspace and configure its Python environment.

In [53]:
# Get reference to the workspace
ws = Workspace.from_config("./ws_config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Set workspace's environment
env = Environment.from_pip_requirements(name = "covid-cxr_env", file_path = "./../requirements.txt")
env.register(workspace=ws)
runconfig = RunConfiguration(conda_dependencies=env.python.conda_dependencies)
print(env.python.conda_dependencies.serialize_to_string())

# Move AML ignore file to root folder
aml_ignore_path = shutil.copy('./.amlignore', './../.amlignore') 

covid-cxr
res_grp1
canadacentral
52f01a1a-c766-41a4-9062-a73cffbef727
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - pyyaml==5.2
  - matplotlib==3.1.1
  - tqdm==4.40.2
  - opencv_python==4.1.0.25
  - tensorflow_gpu==2.0.1
  - dill==0.3.0
  - scikit_image==0.15.0
  - numpy==1.17.4
  - pandas==0.25.3
  - pydicom==1.4.2
  - imbalanced_learn==0.6.1
  - tensorboard==2.0.2
  - lime==0.1.1.37
channels:
- anaconda
- conda-forge



### Create references to persistent and intermediate data
Create DataReference objects that point to our raw data on the blob. Configure a PipelineData object to point to preprocessed images stored on the blob.

In [54]:
# Get the blob datastore associated with this workspace
blob_store = Datastore(ws, name='covidcxrdata0')

# Create data references to folders on the blob
covid_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="raw_covid_data",
    path_on_datastore="data/covid-chestxray-dataset/")
rsna_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="raw_rsna_data",
    path_on_datastore="data/rsna/")
training_logs_dr = DataReference(
    datastore=blob_store,
    data_reference_name="training_logs_data",
    path_on_datastore="logs/training/")
models_dr = DataReference(
    datastore=blob_store,
    data_reference_name="models_data",
    path_on_datastore="models/")

# Set up references to pipeline data (intermediate pipeline storage).
processed_pd = PipelineData(
    "processed_data",
    datastore=blob_store,
    output_name="processed_data")

### Compute Target
Specify and configure the compute target for this workspace. If a compute cluster by the name we specified does not exist, create a new compute cluster.

In [55]:
ct_name = "d2-cpu-cluster"   # Name of our compute cluster
vm_size = "STANDARD_D2_V2"   # Specify the Azure VM for execution of our pipeline

# Set up the compute target for this
try:
    compute_target = AmlCompute(ws, ct_name)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating new compute target")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = 1, 
                                                                max_nodes = 3)    
    compute_target = ComputeTarget.create(ws, ct_name, provisioning_config)  # Create the compute cluster
    
    # Wait for cluster to be provisioned
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20) 
    
print("Azure Machine Learning Compute attached")
print("Compute targets: ", ws.compute_targets)
compute_target = ws.compute_targets[ct_name]

Found existing compute target.
Azure Machine Learning Compute attached
Compute targets:  {'my-gpu-cluster': AmlCompute(workspace=Workspace.create(name='covid-cxr', subscription_id='52f01a1a-c766-41a4-9062-a73cffbef727', resource_group='res_grp1'), name=my-gpu-cluster, id=/subscriptions/52f01a1a-c766-41a4-9062-a73cffbef727/resourceGroups/res_grp1/providers/Microsoft.MachineLearningServices/workspaces/covid-cxr/computes/my-gpu-cluster, type=AmlCompute, provisioning_state=Failed, location=canadacentral, tags=None), 'nc6-gpu-cluster': AmlCompute(workspace=Workspace.create(name='covid-cxr', subscription_id='52f01a1a-c766-41a4-9062-a73cffbef727', resource_group='res_grp1'), name=nc6-gpu-cluster, id=/subscriptions/52f01a1a-c766-41a4-9062-a73cffbef727/resourceGroups/res_grp1/providers/Microsoft.MachineLearningServices/workspaces/covid-cxr/computes/nc6-gpu-cluster, type=AmlCompute, provisioning_state=Failed, location=canadacentral, tags=None), 'nc6s-gpu-cluster': AmlCompute(workspace=Workspace.

### Define pipeline and submit experiment.
Define the steps of an Azure machine learning pipeline. Create an Azure Experiment that will run our pipeline. Submit the experiment to the execution environment.

In [56]:
# Define preprocessing step the ML pipeline
step1 = PythonScriptStep(name="preprocess_step",
                         script_name="azure/preprocess_step.py",
                         arguments=["--coviddatadir", covid_data_dr, "--rsnadatadir", rsna_data_dr, "--preprocesseddir", 
                                    processed_pd],
                         inputs=[covid_data_dr, rsna_data_dr],
                         outputs=[processed_pd],
                         compute_target=compute_target, 
                         source_directory="./../",
                         runconfig=runconfig,
                         allow_reuse=True)

# Define training step in the ML pipeline
est = Estimator(source_directory='./../',
                   script_params=None,
                   compute_target=compute_target,
                   entry_script='azure/train_step.py',
                   pip_packages=['tensorflow', 'tensorboard', 'pandas', 'dill', 'numpy', 'imblearn'],
                   pip_requirements_file='./requirements.txt')
step2 = EstimatorStep(name="estimator_train", 
                      estimator=est, 
                      estimator_entry_script_arguments=["--preprocesseddir", processed_pd, "--traininglogsdir", training_logs_dr,
                                                       "--modelsdir", models_dr],
                      runconfig_pipeline_params=None, 
                      inputs=[processed_pd, training_logs_dr, models_dr], 
                      outputs=[], 
                      compute_target=compute_target)

# Construct the ML pipeline from the steps
steps = [step1, step2]
single_train_pipeline = Pipeline(workspace=ws, steps=steps)
single_train_pipeline.validate()

# Define a new experiment and submit a new pipeline run to the compute target.
experiment = Experiment(workspace=ws, name='SingleTrainExperiment_v2')
experiment.submit(single_train_pipeline, regenerate_outputs=False)
print("Pipeline is submitted for execution")

# Move AML ignore file back to original folder
aml_ignore_path = shutil.move(aml_ignore_path, './.amlignore') 

Step preprocess_step is ready to be created [d37678aa]
Step estimator_train is ready to be created [30be703d]
Created step preprocess_step [d37678aa][81516a06-73ee-4cf2-b7cc-ed3a07e09624], (This step will run and generate new outputs)
Created step estimator_train [30be703d][393c7bd8-7f8f-4618-ad5d-bc165390b0cb], (This step will run and generate new outputs)
Using data reference raw_covid_data for StepId [fcc71f99][e504c00e-cf0c-408e-8bc5-ebe6819f8135], (Consumers of this data are eligible to reuse prior runs.)
Using data reference raw_rsna_data for StepId [08ad5039][89ae9f7a-275f-42f8-aec8-f22978bb298c], (Consumers of this data are eligible to reuse prior runs.)
Using data reference training_logs_data for StepId [b15ee1a9][23cf5660-b8cf-49f1-a90f-f805c3927088], (Consumers of this data are eligible to reuse prior runs.)
Using data reference models_data for StepId [7cf87833][7e02318c-dbf3-4f78-badd-685e749f72f9], (Consumers of this data are eligible to reuse prior runs.)
Submitted Pipeli